In [1]:
!pip install ultralytics --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.1 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO
from google.colab import files
import os
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [30]:
uploaded = files.upload()
input_path = list(uploaded.keys())[0]


Saving person2.mp4 to person2 (1).mp4


In [4]:
model = YOLO('yolov5s.pt')


PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 17.7M/17.7M [00:00<00:00, 81.7MB/s]


In [33]:
import cv2
import time
import numpy as np
import uuid

# Helper functions
def iou(boxA, boxB):
    """Compute Intersection over Union (IoU) between two boxes"""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return interArea / float(boxAArea + boxBArea - interArea + 1e-6)

def create_kalman(cx, cy):
    kf = cv2.KalmanFilter(4, 2)
    kf.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]], dtype=np.float32)
    kf.measurementMatrix = np.eye(2, 4, dtype=np.float32)
    kf.processNoiseCov = 1e-2 * np.eye(4, dtype=np.float32)
    kf.measurementNoiseCov = 1e-1 * np.eye(2, dtype=np.float32)
    kf.statePost = np.array([[cx], [cy], [0], [0]], dtype=np.float32)
    return kf

# Tracker data
trackers = {}  # ID -> {kalman, last_bbox, frames_since_seen}

# Parameters
MAX_LOST = 15  # remove object if not seen in N frames

# Video setup
cap = cv2.VideoCapture(input_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_path = 'tracked_multi.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

frame_id = 0
class_names = model.names

while True:
    start_time = time.time()
    success, frame = cap.read()
    if not success:
        break

    results = model.track(source=frame, persist=True, verbose=False, imgsz=960, conf=0.25, iou=0.45)

    # Filter person detections only
    filtered_boxes = [
        box for box in results[0].boxes
        if int(box.cls.item()) == 0
    ]

    detections = []
    for box in filtered_boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        detections.append((x1, y1, x2, y2, cx, cy))

    # Match detections to existing trackers
    assigned = set()
    updated_trackers = {}

    for track_id, data in trackers.items():
        kf, bbox, lost = data['kf'], data['bbox'], data['lost']
        prediction = kf.predict()
        pred_cx, pred_cy = int(prediction[0]), int(prediction[1])

        best_iou = 0
        best_match = None

        for i, (x1, y1, x2, y2, cx, cy) in enumerate(detections):
            if i in assigned:
                continue
            iou_score = iou(bbox, (x1, y1, x2, y2))
            if iou_score > best_iou:
                best_iou = iou_score
                best_match = (i, (x1, y1, x2, y2, cx, cy))

        if best_iou > 0.3 and best_match:
            i, (x1, y1, x2, y2, cx, cy) = best_match
            kf.correct(np.array([[np.float32(cx)], [np.float32(cy)]]))
            updated_trackers[track_id] = {'kf': kf, 'bbox': (x1, y1, x2, y2), 'lost': 0}
            assigned.add(i)
        else:
            # Keep old prediction if no match
            updated_trackers[track_id] = {'kf': kf, 'bbox': bbox, 'lost': lost + 1}

    # Add new trackers for unmatched detections
    for i, (x1, y1, x2, y2, cx, cy) in enumerate(detections):
        if i in assigned:
            continue
        new_id = str(uuid.uuid4())[:8]
        kf = create_kalman(cx, cy)
        updated_trackers[new_id] = {'kf': kf, 'bbox': (x1, y1, x2, y2), 'lost': 0}

    # Remove lost trackers
    trackers = {tid: data for tid, data in updated_trackers.items() if data['lost'] <= MAX_LOST}

    # Draw
    annotated_frame = frame.copy()
    for tid, data in trackers.items():
        x1, y1, x2, y2 = data['bbox']
        prediction = data['kf'].predict()
        pred_cx, pred_cy = int(prediction[0]), int(prediction[1])

        # Draw red bounding box and label
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        label = f"ID: {tid}"
        cv2.putText(annotated_frame, label, (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.circle(annotated_frame, (pred_cx, pred_cy), 5, (0, 0, 255), -1)

    # FPS overlay
    end_time = time.time()
    processing_fps = 1 / (end_time - start_time + 1e-6)
    cv2.putText(annotated_frame, f"FPS: {processing_fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    out.write(annotated_frame)
    frame_id += 1

cap.release()
out.release()


/tmp/ipython-input-33-1845732358.py:110: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_cx, pred_cy = int(prediction[0]), int(prediction[1])
/tmp/ipython-input-33-1845732358.py:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_cx, pred_cy = int(prediction[0]), int(prediction[1])


In [35]:
from google.colab import files
files.download('tracked_multi.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>